# Import ResOpsUs data

In [1]:
# import modules
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import geopandas as gpd
import cartopy.crs as ccrs
import os
import utils
import seaborn as sns
import hydroeval as he
import warnings
import pickle

warnings.filterwarnings('ignore')

# plot settings
utils.set_plot_param()

In [13]:
df_attributes = pd.read_csv(obsdir_resopsus+'attributes/reservoir_attributes.csv')
df_attributes

gdf = gpd.GeoDataFrame(
    df_attributes, geometry=gpd.points_from_xy(df_attributes.LONG, df_attributes.LAT))

gdf.to_file(obsdir_resopsus+'ResOpsUS_attributes.shp')

In [4]:
df_timeseries_variables = pd.read_csv(obsdir_resopsus+'attributes/time_series_variables.csv')
df_timeseries_variables

,Variable,Unit,Unit Text,Time Period,Parameter
0,IN,ms,cubic meters per second,Daily,Reservoir Inflow
1,STOR,MCM,Million Cubic Meters,Daily,Reservoir Storage
2,ELE,m,meters,Daily,Reservoir Elevation
3,OUT,ms,cubic meters per second,Daily,Reservoir Outflow
4,EVAP,MCM,million cubic meters,Daily,Reservoir Evaporation


In [10]:
df_timeseries_inventory = pd.read_csv(obsdir_resopsus+'attributes/time_series_inventory.csv')
df_inventory = df_timeseries_inventory[df_timeseries_inventory['OUTFLOW']==1][df_timeseries_inventory['STORAGE']==1]
df_inventory

,DAM_ID,STORAGE,STORAGE_START,STORAGE_END,DATA_SOURCE,INFLOW,INFLOW_START,INFLOW_END,DATA_SOURCE.1,OUTFLOW,...,OUTFLOW_END,DATA_SOURCE.2,ELEVATION,ELEVATION_START,ELEVATION_END,DATA_SOURCE.3,EVAPORATION,EVAPORATION_START,EVAPORATION_END,DATA_SOURCE.4
0,41,1,10/1/70,8/31/20,SCL,1,10/1/70,8/31/20,SCL,1,...,8/31/20,SCL,1,10/1/70,8/31/20,SCL,0,NaN,NaN,NaN
1,42,1,10/1/70,8/31/20,SCL,1,10/1/70,8/31/20,SCL,1,...,8/31/20,SCL,1,10/1/70,8/31/20,SCL,0,NaN,NaN,NaN
2,54,1,10/1/15,12/31/20,BOR_PN,0,NaN,NaN,NaN,1,...,12/31/20,BOR_PN,0,NaN,NaN,NaN,0,NaN,NaN,NaN
3,55,1,1/1/65,12/31/20,BOR_PN,1,1/1/65,12/31/20,BOR_PN,1,...,12/31/20,BOR_PN,0,NaN,NaN,NaN,0,NaN,NaN,NaN
4,56,1,1/1/90,12/31/20,ACE_SEA,0,NaN,NaN,NaN,1,...,12/31/20,usgs_12105900,1,1/1/90,12/31/20,ACE_SEA,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673,7311,1,10/1/79,9/30/15,DUKE,1,10/1/79,9/30/15,DUKE,1,...,9/30/15,DUKE,1,10/1/79,9/30/15,DUKE,1,10/1/79,9/30/15,DUKE
674,7313,1,3/27/53,12/31/18,BOR,1,3/28/53,12/31/18,BOR,1,...,12/31/18,BOR,1,3/27/53,12/31/18,BOR,0,NaN,NaN,NaN
675,7317,1,1/1/69,12/6/19,TVA,0,NaN,NaN,NaN,1,...,12/6/19,TVA,1,1/2/69,12/6/19,TVA,0,NaN,NaN,NaN
676,7318,1,4/1/68,11/17/19,ACE_NAS,1,4/2/68,11/17/19,ACE_NAS,1,...,11/17/19,ACE_NAS,1,4/1/68,11/17/19,ACE_NAS,0,NaN,NaN,NaN


In [14]:
obsdir = '/glade/work/ivanderk/data/ResOpsUS/'



for dam_id in df_inventory[['DAM_ID']].values[0]: 
    df_usresobs = pd.read_csv(obsdir+'time_series_all/ResOpsUS_'+str(int(dam_id))+'.csv')
    
    # get metadata from attributes folder, this is similar to GRanD
    dam_name = df_attributes.loc[df_attributes['DAM_ID'] == dam_id,'DAM_NAME']
    dam_lon  = df_attributes.loc[df_attributes['DAM_ID'] == dam_id,'LONG']
    dam_lat  = df_attributes.loc[df_attributes['DAM_ID'] == dam_id,'LAT']

In [15]:

df_usresobs


,date,storage,inflow,outflow,elevation,evaporation
0,1970-10-01,1149.749,29.733,NaN,485.269,NaN
1,1970-10-02,1146.835,26.901,NaN,485.205,NaN
2,1970-10-03,1148.499,20.530,NaN,485.242,NaN
3,1970-10-04,1150.026,26.193,NaN,485.275,NaN
4,1970-10-05,1145.588,37.520,NaN,485.178,NaN
...,...,...,...,...,...,...
18228,2020-08-27,1279.995,31.856,39.360,488.067,NaN
18229,2020-08-28,1277.524,28.657,57.370,488.015,NaN
18230,2020-08-29,1277.088,28.543,35.169,488.006,NaN
18231,2020-08-30,1273.751,27.779,66.375,487.936,NaN
